In [1]:
from ollama import chat
from ollama import ChatResponse
import pandas as pd
import re
import json

In [87]:
def llm_classifier_single_value(message: str, symptoms_codes: list, batch: dict) -> str:
    # response = chat(model="qwen2.5:3b", messages=[
    #     {
    #         'role': 'user',
    #         'content': f'Based on the provided list of symtoms {symptoms}, I want to classify, with one or more of the classes present in the list, the following sentence: \"{message}\", without any words, just a plain list the predictions. Here are some predefined sentences for fine-tuning:{df}'
    #     }
    # ])
#     response = chat(model="qwen3:4b", messages=[
#     {
#         "role": "user",
#         "content": (
#             f"You are an AI model trained to classify medical symptoms. "
#             f"Given a list of possible classes {symptoms_codes} and a new sentence: \"{message}\", "
#             f"predict the relevant classes from the list. Only return a JSON array with the predicted class names, "
#             f"without any explanation, formatting, or extra text.\n"
#             f"Here are some example training sentences and their labels for context:\n{batch}\n\n"
#             # f"As a medical AI, return classes that you are very sure about\n\n"
#             f"Your answer must be a valid Python list of strings."
#         )
#     }
# ])
    response = chat(model="qwen3:4b", messages=[
    {
        "role": "user",
        "content": (
            f"You are a strict symptom classifier.\n"
            f"You are given:\n- A sentence describing symptoms: \"{message}\"\n"
            f"A list of valid symptom codes: {symptoms_codes}\n\n"
            f"Your job is to return a Python list of symptom codes that are clearly and directly indicated in the sentence.\n"
            f"Do NOT infer or guess the cause of the symptoms.\n"
            f"Do NOT include codes that are not explicitly or semantically mentioned.\n"
            f"You ARE allowed to match equivalent everyday phrases, e.g., 'high temperature' → 'fever', 'throwing up' → 'vomiting'.\n"
            f"Only return the most direct and clearly stated symptoms you are very sure about.\n\n"
            f"Here are some labeled examples:\n{batch}\n\n"
            f"Output a valid Python list of symptom codes, and nothing else."
        )
    }
])
    return response.message.content

In [88]:
no_values_data = {}
no_value_meaning_data = {}
multiple_values_data = {}

In [89]:
with open("evidences/release_evidences_simplified_no_values_examples.json", "r") as file:
    no_values_data = json.load(file)

with open("evidences/release_evidences_simplified_no_value_meaning_examples.json", "r") as file:
    no_value_meaning_data = json.load(file)

with open("evidences/release_evidences_simplified_value_meaning_examples.json", "r") as file:
    multiple_values_data = json.load(file)

In [90]:
message = "I noticed an intense fever, coughing non-stop. I noticed skin rash all over my body"

In [91]:
len(no_values_data)

208

In [94]:
import ast

In [95]:
batch = {}
list_of_values = list(no_values_data.keys())[0:50]
for i in range(len(list_of_values)):
    batch[list_of_values[i]] = no_values_data[list_of_values[i]]
print(list_of_values)
print(batch)
strstr = llm_classifier_single_value(message, list_of_values, batch)

after_think = strstr.split("</think>")[-1].strip()

# Evaluăm lista ca obiect Python real
try:
    result_list = ast.literal_eval(after_think)
    print(result_list)          # ['E_58_@_10']
    print(type(result_list))    # <class 'list'>
except Exception as e:
    print("Eroare la parsare:", e)

['E_91', 'E_53', 'E_159', 'E_129', 'E_154', 'E_155', 'E_210', 'E_140', 'E_51', 'E_75', 'E_89', 'E_114', 'E_82', 'E_148', 'E_94', 'E_220', 'E_161', 'E_179', 'E_162', 'E_173', 'E_33', 'E_218', 'E_93', 'E_66', 'E_163', 'E_30', 'E_127', 'E_181', 'E_88', 'E_43', 'E_156', 'E_144', 'E_216', 'E_201', 'E_217', 'E_215', 'E_64', 'E_96', 'E_16', 'E_50', 'E_97', 'E_9', 'E_76', 'E_102', 'E_65', 'E_74', 'E_205', 'E_63', 'E_128', 'E_190']
{'E_91': 'you have a fever (either felt or measured with a thermometer)', 'E_53': 'you have pain somewhere, related to your reason for consulting', 'E_159': 'you lose consciousness', 'E_129': 'you have any lesions, redness or problems on your skin that you believe are related to the condition you are consulting for', 'E_154': 'your skin much paler than usual', 'E_155': 'you feel your heart is beating fast (racing), irregularly (missing a beat) or do you feel palpitations', 'E_210': 'you recently thrown up blood or something resembling coffee beans', 'E_140': 'you rec

In [61]:
def llm_classifier_with_values(message: str, symptoms_with_scales: dict) -> str:
    response = chat(model="qwen3:4b", messages=[
        {
            "role": "user",
            "content": (
                f"You are a medical AI trained to detect and quantify symptom severity from natural language.\n\n"
                f"Input sentence: \"{message}\"\n\n"
                f"You are given a list of symptom codes with associated questions and scales (0 to 10):\n"
                f"{symptoms_with_scales}\n\n"
                f"Your job:\n"
                f"1. Identify which symptoms from the list are explicitly or semantically mentioned in the sentence.\n"
                f"2. Estimate the intensity of each symptom on the 0–10 scale, based on words like 'mild', 'severe', etc.\n"
                f"3. Return only clearly expressed symptoms.\n\n"
                f"Output format:\n"
                f"A valid Python list of strings, where each string is in the format: 'E_CODE_@_VALUE', for example: [\"E_136_@_9\", \"E_56_@_7\"]\n\n"
                f"Rules:\n"
                f"- DO NOT invent symptoms.\n"
                f"- DO NOT include explanations.\n"
                f"- DO NOT output anything other than the list.\n"
                f"- ONLY include symptoms from the given dictionary.\n"
                f"- If no matching symptom is present, return an empty list: []"
            )
        }
    ])
    return response.message.content

In [9]:
message_for_value = "I have pain that appeared some fast, while the pain is not as intese, maybe mild. I have pain from rash very extreme."

In [10]:
print(llm_classifier_with_values(message_for_value, no_value_meaning_data))

<think>
Okay, let's tackle this. The user provided a sentence and a list of symptoms with their respective scales. My job is to identify which symptoms are mentioned and assign the correct intensity values.

First, I'll read the input sentence again: "I have pain that appeared some fast, while the pain is not as intese, maybe mild. I have pain from rash very extreme."

Looking at the symptoms in the list:

E_59: How fast did the pain appear? The sentence says "appeared some fast," which probably refers to the speed of onset. So that's E_59. The scale is 0-10. The word "fast" here might mean the pain started quickly, so maybe a higher value. But the example is about speed, so I need to map "some fast" to a number. The scale is from 0 to 10, but the example isn't clear. Wait, the example is "How fast did the pain appear?" So the scale is for the speed, not the intensity. The user says "some fast"—maybe "some" is a modifier. But the scale is 0-10, so maybe "fast" is a 7 or 8? Or maybe it'

In [60]:
def llm_classifier_multiple_value_meaning(message: str, symptoms_codes: list, batch: dict) -> str:
    # response = chat(model="qwen2.5:3b", messages=[
    #     {
    #         'role': 'user',
    #         'content': f'Based on the provided list of symtoms {symptoms}, I want to classify, with one or more of the classes present in the list, the following sentence: \"{message}\", without any words, just a plain list the predictions. Here are some predefined sentences for fine-tuning:{df}'
    #     }
    # ])
#     response = chat(model="qwen3:4b", messages=[
#     {
#         "role": "user",
#         "content": (
#             f"You are an AI model trained to classify medical symptoms. "
#             f"Given a list of possible classes {symptoms_codes} and a new sentence: \"{message}\", "
#             f"predict the relevant classes from the list. Only return a JSON array with the predicted class names, "
#             f"without any explanation, formatting, or extra text.\n"
#             f"Here are some example training sentences and their labels for context:\n{batch}\n\n"
#             # f"As a medical AI, return classes that you are very sure about\n\n"
#             f"Your answer must be a valid Python list of strings."
#         )
#     }
# ])
    response = chat(model="qwen3:4b", messages=[
    {
        "role": "user",
        "content": (
            f"You are a strict symptom classifier.\n"
            f"You are given:\n- A sentence describing symptoms: \"{message}\"\n"
            f"A list of valid symptom codes: {symptoms_codes}\n\n"
            f"Your job is to return a Python list of symptom codes that are clearly and directly indicated in the sentence.\n"
            f"Do NOT infer or guess the cause of the symptoms.\n"
            f"Do NOT include codes that are not explicitly or semantically mentioned.\n"
            f"You ARE allowed to match equivalent everyday phrases, e.g., 'high temperature' → 'fever', 'throwing up' → 'vomiting'.\n"
            f"If the sentence includes right (R) or left (L), be precise and match it with the appropriate code. Do not confuse them.\n"
            f"Do not consider a location that is not explicitly mentioned in the sentence as valid.\n"
            f"Only return the most direct and clearly stated symptoms you are very sure about.\n\n"
            f"Here are some labeled examples:\n{batch}\n\n"
            f"Output a valid Python list of symptom codes, and nothing else."
        )
    }
])
    return response.message.content



In [12]:
message_value_meaning = "I feel pain in my left arm, but also in anus and my reight leg."

In [13]:
batch = {}
list_of_values = list(multiple_values_data.keys())[0:50]
for i in range(len(list_of_values)):
    batch[list_of_values[i]] = multiple_values_data[list_of_values[i]]
print(llm_classifier_multiple_value_meaning(message_value_meaning, list_of_values, batch))

<think>
Okay, let me try to figure out how to approach this. The user provided a bunch of JSON data with symptom codes and their descriptions. They want a valid Python list of symptom codes, nothing else.

First, I need to parse the JSON structure. The main keys are the question IDs like 'E_204', 'E_203', etc. Each has an 'example' and 'codes' list. The task is to extract all the code values from each 'codes' list.

Wait, but looking at the example, for E_204, the codes include 'V_10(North Africa)' etc. But the user says "symptom codes" so maybe just the code part, like 'V_10', 'V_0', etc. But the example shows that the codes have descriptions in parentheses. However, the user's instruction says "symptom codes", so maybe the actual code is the part before the parentheses. But I need to check the example given in the problem.

Looking at the first entry: 'V_10(North Africa)' — but in the output, maybe we just need the code part, like 'V_10', 'V_0', etc. However, the user's example in th

In [59]:
def llm_classifier_with_value_meaning(message: str, symptoms_with_values: dict) -> str:
    response = chat(model="qwen3:4b", messages=[
        {
            "role": "user",
            "content": (
                f"You are a medical AI trained to identify symptom-related information from natural language.\n\n"
                f"You are given a dictionary where each key is a symptom code (E_CODE), associated with an example and a list of possible values, each with a code and a description:\n\n"
                f"{symptoms_with_values}\n\n"
                f"Your task:\n"
                f"1. Detect which E_CODEs are relevant to the input message based on semantic meaning.\n"
                f"2. For each relevant E_CODE, choose the most appropriate V_CODEs(can be more than just one, depending on context) (based on the value descriptions).\n"
                f"3. Return the answer in this exact format:\n"
                f"A Python list of strings: [\"E_CODE_@_V_CODE\", ...]\n\n"
                f"The input message is: \"{message}\"\n\n"
                f"Rules:\n"
                f"- DO NOT invent E_CODEs or V_CODEs.\n"
                f"- ONLY use values from the provided dictionary.\n"
                f"- DO NOT add explanations or extra text.\n"
                f"You ARE allowed to match equivalent everyday phrases, e.g., 'high temperature' → 'fever', 'throwing up' → 'vomiting'.\n"
                f"- If nothing matches clearly, return: []"
            )
        }
    ])
    return response.message.content

In [22]:
message_value_meaning = "I feel pain in my both arms. I feel pain under the jaw. Also, this pain radiates to my lower chest."

In [23]:
batch = {}
list_of_values = list(multiple_values_data.keys())[0:2]
for i in range(len(list_of_values)):
    batch[list_of_values[i]] = multiple_values_data[list_of_values[i]]
#print(batch)
print(llm_classifier_with_value_meaning(message_value_meaning, batch))

<think>
Okay, let's tackle this. The user says they feel pain in both arms, under the jaw, and it radiates to the lower chest.

First, I need to find the relevant E_CODEs. The input mentions pain in both arms and radiates to the lower chest. Looking at the E_CODEs provided, there's E_CODE for "pain in" and "pain radiates to". Wait, the E_CODEs are "pain in" and "pain radiates to". Wait, the keys are "E_CODEs" but the user provided two entries: one for "pain in" (E_CODE) and another for "pain radiates to" (E_CODE). Wait, looking at the user's input, the two E_CODEs are "E_CODE" with the first being "pain in" and the second being "pain radiates to". Wait, the user's input is the two E_CODEs: "pain in" and "pain radiates to". Wait, the problem says that the user's input is the message, and we need to detect which E_CODEs are relevant based on semantic meaning. So, the E_CODEs are the keys in the provided dictionary. Wait, the user provided two E_CODEs: "pain in" and "pain radiates to". Wa

In [31]:
multiple_values_data.keys()

dict_keys(['E_55', 'E_57', 'E_54', 'E_133', 'E_130', 'E_135', 'E_131', 'E_152', 'E_204'])

In [33]:
e_codes_no_location = ['E_54', 'E_130', 'E_135', 'E_131', 'E_204']
print(multiple_values_data[e_codes_no_location[0]])
print(multiple_values_data[e_codes_no_location[1]])
print(multiple_values_data[e_codes_no_location[2]])
print(multiple_values_data[e_codes_no_location[3]])
print(multiple_values_data[e_codes_no_location[4]])


{'example': 'Your pain is', 'codes': ['V_11(NA)', 'V_71(heartbreaking)', 'V_112(haunting)', 'V_154(tedious)', 'V_161(sensitive)', 'V_179(a knife stroke)', 'V_180(tugging)', 'V_181(burning)', 'V_182(a cramp)', 'V_183(heavy)', 'V_184(a pulse)', 'V_191(violent)', 'V_192(sharp)', 'V_193(sickening)', 'V_196(scary)', 'V_198(exhausting)']}
{'example': 'The color of the rash is', 'codes': ['V_11(NA)', 'V_86(dark)', 'V_107(yellow)', 'V_138(pale)', 'V_156(pink)', 'V_157(red)']}
{'example': 'The lesion(or lesions if are multiple) are larger than 1cm(Y for yes and N for no)', 'codes': ['V_10(N)', 'V_12(Y)']}
{'example': 'Your lesion(or lesions) peel off(Y for yes and N for no)', 'codes': ['V_10(N)', 'V_12(Y)']}
{'example': 'If you traveled out of the country in the last 4 weeks, you traveled in', 'codes': ['V_10(N)', 'V_0(North Africa)', 'V_1(West Africa)', 'V_2(South Africa)', 'V_3(Central America)', 'V_4(North America)', 'V_5(South America)', 'V_6(Asia)', 'V_7(South East Asia)', 'V_8(Caraibes)',

In [29]:
e_codes = ['E_55', 'E_57', 'E_133', 'E_152']
print(len(multiple_values_data[e_codes[0]]['codes']))
print(len(multiple_values_data[e_codes[1]]['codes']))
print(len(multiple_values_data[e_codes[2]]['codes']))
print(len(multiple_values_data[e_codes[3]]['codes']))
list_e_55 = multiple_values_data[e_codes[0]]['codes']
list_e_57 = multiple_values_data[e_codes[1]]['codes']
list_e_133 = multiple_values_data[e_codes[2]]['codes']
list_e_152 = multiple_values_data[e_codes[3]]['codes']

for elem in zip(list_e_55, list_e_57, list_e_133, list_e_152):
    if elem[0] == elem[1] == elem[2] == elem[3]:
        continue
    else:
        print("Not compatible")

165
165
165
165


In [121]:
def build_prompt(message, e_codes_dict, v_codes_list):
    e_code_str = "\n".join([f"{code} → \"{info['example']}\"" for code, info in e_codes_dict.items()])
    v_code_str = "\n".join(v_codes_list)

    return f"""
You are a medical AI assistant trained to extract structured symptom information from patient descriptions.

You are given:
- A list of symptom types (E_CODEs), each with a short example that defines its meaning.
- A shared list of body part locations (V_CODEs) to choose from.

---

Task:
1. Carefully read the input message.
2. Check if **any** of the E_CODE examples clearly match the meaning of parts of the message.
3. For **each matching E_CODE**, choose all relevant V_CODEs that describe **exactly** the mentioned body part locations.
4. Only select a V_CODE if the message refers **explicitly and unambiguously** to that body part.
   - **Do not guess** or infer based on context.
   - **If the reference is not completely clear, skip that V_CODE.**
5. If there is **no clear match** for an E_CODE, skip it entirely.
6. Only use E_CODEs and V_CODEs from the lists below. Do not invent anything.

---

E_CODEs and their semantic meaning (via example):
{e_code_str}

---

V_CODEs (shared across all E_CODEs):
{v_code_str}

---

Input message:
\"{message}\"

---

Output rules:
- Format: ["E_CODE_@_V_CODE", ...]
- IMPORTANT: Use **exactly** "_@_" (with underscores). This is required.
- ✅ Correct example: ["E_55_@_V_27", "E_133_@_V_163"]
- ❌ Incorrect formats: ["E_55@V_27"], ["E_55-V_27"], ["E_CODE_E_55_V_27"]
- Only include E_CODE_@_V_CODE pairs if both match clearly.
- If nothing matches, return: []
- Do not include explanations, comments, or any other text. Just the list.
"""

In [122]:
def llm_classifier_with_value_localization(message: str) -> str:
    response = chat(model="qwen3:4b", messages=[
        {
            'role': 'user',
            'content': message         
        }
    ])
    return response.message.content

In [123]:
localization_values = {code: multiple_values_data[code] for code in e_codes}
v_codes = multiple_values_data[e_codes[0]]["codes"]

string_prompt = build_prompt(message_value_meaning, localization_values, v_codes)
aux = llm_classifier_with_value_localization(string_prompt)
after_think = aux.split("</think>")[-1].strip()
print(after_think)

# Evaluăm lista ca obiect Python real
try:
    result_list = ast.literal_eval(after_think)
    print(result_list)          # ['E_58_@_10']
    print(type(result_list))    # <class 'list'>
except Exception as e:
    print("Eroare la parsare:", e)

["E_55_@_V_27", "E_55_@_V_28", "E_55_@_V_163", "E_57_@_V_55", "E_57_@_V_56"]
['E_55_@_V_27', 'E_55_@_V_28', 'E_55_@_V_163', 'E_57_@_V_55', 'E_57_@_V_56']
<class 'list'>


In [68]:
def build_single_e_code_prompt(message: str, e_code: str, example: str, v_codes: list[str]) -> str:
    v_code_str = "\n".join(v_codes)

    return f"""
You are a medical AI assistant trained to extract structured symptom information from patient descriptions.

You are analyzing a single symptom type identified by an E_CODE.

---

E_CODE: {e_code}  
Meaning: {example}

Available values (V_CODEs) for this symptom type:
{v_code_str}

---

Task:
1. Read the input message.
2. Determine whether this E_CODE is relevant to the message based on its meaning.
3. If it is relevant, select ALL V_CODEs from the list that are clearly referred to in the message.
    A match is valid if the message contains:
   - a direct statement (e.g., "Yes", "No"), or
   - unambiguous descriptive language (e.g., "small lesion"), or
   - negative or comparative logic (e.g., "not larger than 1 cm", "less than 1 cm", "does not exceed 1 cm")
   - You may use basic commonsense reasoning to interpret clear, everyday language.
   - Do NOT select a V_CODE if the meaning is vague or ambiguous.
4. If the E_CODE does not apply, or none of the V_CODEs match clearly, return an empty list.

---

Input message:
\"{message}\"

---

Output format:
- List of matched value codes in this format: ["{e_code}_@_V_CODE", ...]
- If no values match clearly, return: []
- Do not include any explanation or extra text.
"""

In [81]:
message_one_code = "I couldn't peel off my lesions"
code = multiple_values_data['E_131']
prompt = build_single_e_code_prompt(message_one_code, 'E_131', code['example'], code['codes'])

In [82]:
aux = llm_classifier_with_value_localization(prompt)
print(aux)

<think>
Okay, let's tackle this. The E_CODE is E_131, which asks if the lesion or lesions peel off. The available V_CODEs are V_10 (N) and V_12 (Y). The input message is "I couldn't peel off my lesions".

First, I need to determine if the E_CODE is relevant. The meaning of E_131 is about whether the lesion peels off. The patient's message says they "couldn't peel off my lesions". That directly relates to the question in E_131. 

Now, checking the V_CODEs. The options are Y (yes) and N (no). The message says "couldn't peel off", which implies that the peeling didn't happen. So that would correspond to "No", which is V_10. But wait, the V_CODEs are V_10 (N) and V_12 (Y). So if the patient couldn't peel off, that's a negative answer. Therefore, the correct V_CODE here is V_10. 

But wait, the message says "couldn't peel off", which is a direct negative statement. So the answer should be N. Therefore, the matched value code is V_10. 

I need to make sure there's no ambiguity. The message i

In [115]:
def build_multi_e_code_prompt(message: str, e_code_data: dict) -> str:
    e_code_sections = []
    for e_code, e_info in e_code_data.items():
        example = e_info["example"]
        v_codes = "\n".join(e_info["codes"])
        section = f"E_CODE: {e_code}\nMeaning: {example}\nAvailable V_CODEs:\n{v_codes}"
        e_code_sections.append(section)

    e_code_str = "\n\n".join(e_code_sections)

    return f"""
You are a medical AI assistant trained to extract structured symptom information from patient descriptions.

Each symptom type is identified by an E_CODE. Each E_CODE has specific possible values called V_CODEs.

---

Your Task:
1. For each E_CODE, check if it applies to the message.
2. If it does, extract one or more matching V_CODEs.
3. Output each match in **this exact format**: E_CODE_@_V_CODE
   - Example: E_01_@_V_09
   - DO NOT use any other format like E_CODE_E_01_V_09 (this is INVALID)
   - DO NOT include extra prefixes or suffixes.
   - DO NOT use just @ — it must be exactly "_@_"
4. If nothing matches, return an empty list: []

---

E_CODE definitions and values:
{e_code_str}

---

Input message:
\"{message}\"

---

Output:
- List of strings formatted exactly like: ["E_CODE_@_V_CODE"]
"""

In [116]:
e_codes_no_location = ['E_54', 'E_130', 'E_135', 'E_131', 'E_204']
e_code_data = {code: multiple_values_data[code] for code in e_codes_no_location}
message_mult_e_codes = 'I went to Europe last week and suddenly I had a pain like a knife stroke'

In [117]:
prompttt = build_multi_e_code_prompt(message_mult_e_codes, e_code_data)

In [118]:
aux = llm_classifier_with_value_localization(prompttt)

after_think = aux.split("</think>")[-1].strip()
print(after_think)

# Evaluăm lista ca obiect Python real
try:
    result_list = ast.literal_eval(after_think)
    print(result_list)          # ['E_58_@_10']
    print(type(result_list))    # <class 'list'>
except Exception as e:
    print("Eroare la parsare:", e)

["E_54_@_V_179", "E_204_@_V_9"]
['E_54_@_V_179', 'E_204_@_V_9']
<class 'list'>
